In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# What's this
- 主にチーム内での情報共有を目的とした文書ですがせっかくなのでpublicにしています
- このNotebookは本コンペの概要と、提供データの各カラムの説明をまとめていきます

# M5 Forecasting
- Walmartで売られている商品の販売台数を予測します
- 過去1913日分のデータを元に将来の28日間の売上台数の予測を行います
- コンペは2段階になっています。終了一ヶ月前までは、d_1からd_1913のデータを元にd_1914からd_1941の28日分の予測を行い、これがパブリックボードになります。
- 終了一ヶ月前にd_1914_d1941の正解データが提供され、これをもとにd_1942からd_1969の28日分の予測を行います。こちらがプライベートボードの結果になります。
- 対象の店舗はカリフォルニア(CA),テキサス(TX)、ウィスコンシン(WI)から計10店舗
- 商品IDは`HOBBIES_1_001`(カテゴリ_種別ID_品目ID)のようにマスクされており、直接商品名を知ることはできません
- 予測結果は商品ID×店舗ID×28日分のデータとして作成します

# Data
- 5つのcsvが提供されます
  - `calendar.csv`
    - 製品が販売された日付に関する情報が含まれています。休日やスーパーボールなどのイベント情報が含まれています
  - `sales_train_validation.csv`
    - 製品および店舗ごとの過去の日単位の販売データが含まれます
  - `sample_submission`
    - サンプルの提出用データ
  - `sell_prices.csv`
    - 店舗および日付ごとに販売される製品の価格に関する情報が含まれています
  - `sales_train_evaluation.csv`
    - コンペの締め切り一ヶ月前に公開されます。d_1914からd_1941の正解データになります。





# sell_prices.csv
- 商品価格
- 店舗、アイテム、週によって一意になります
- 販売開始前のレコードは無いようです

## wm_yr_wk
- calendar.csvで登場しますが
- 土曜日から金曜日の期間で1週間を表します
- 価格改定は必ず土曜日に発生していると予想されます
  - もしかしたら週の途中で価格改定があった場合、週単位に丸められている可能性もあります


In [ ]:
sp = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
sp

In [ ]:
sp.describe()

# calendar.csv

In [ ]:
cl = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
cl

## wm_yr_wk
- 土曜日から金曜日の1週間を表します
- 1月の第1週から開始し、12月の最終週までの連番になります
- {1固定}{西暦の下2桁}{週番号(01-52)}

In [ ]:
cl[cl['wm_yr_wk']==11101]

In [ ]:
cl[cl['wm_yr_wk']==11152]

- 11153は無い

In [ ]:
cl[cl['wm_yr_wk']==11153]

- 2012年はまた01から開始

In [ ]:
cl[cl['wm_yr_wk']==11201]

## event_name, event_type
- 2つのイベントが発生した日はevent_name_2, event_type_2の方に埋められる

In [ ]:
pd.set_option("display.max_rows", 200)
cl[['event_name_1','event_type_1','event_name_2','event_type_2']].dropna(how='all')

In [ ]:
cl.describe()

- snap_CA, snap_TX, snap_WIについて  
  [フードスタンプ](https://ja.wikipedia.org/wiki/%E3%83%95%E3%83%BC%E3%83%89%E3%82%B9%E3%82%BF%E3%83%B3%E3%83%97)
  
  > 形態は、通貨と同様に使用できるバウチャー、金券の一種。一般のスーパーマーケットでも使用できる。対象は食料品であり、タバコやビールなどの嗜好品は対象外となる。
    
  > 2018年9月時点では、約3,858万人、約1,940万世帯、が利用し、米国国民の約8人に1人がSNAPを受給しているという状況になっている。
  
  - 金額は一人あたり100~200ドル程度のようです
  - 州によって配布日が異なるようです
  - FOODカテゴリへの影響が予想されます
  - 転売は違法ですが、現在でも問題視されていることから低価格の嗜好品への影響も予想されます

# sales_train_validation.csv
- 主たるtrainデータ
- 店舗、商品、日毎の売上台数
- 

In [ ]:
stv = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
stv

In [ ]:
stv.describe()

# sample_submission.csv
- 末尾に`_validation`,`_evaluation`がついてる
- `_validation`は前半のパブリックボードで利用される。d_1914~d_1941の28日分
- `_evaluation`は後半のプライベートボードで利用される。d_1942~d_1969の28日分

In [ ]:
smpl = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
smpl

In [ ]:
smpl.describe()